In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [2]:
df = pd.read_csv("C:/동국대학교/공모전/미래에셋/뉴스데이터.csv", encoding='utf-8')

In [3]:
df

,news_title,news_url,news_contents,news_dates
0,"“밈 주식 경계”vs“더 오른다”…‘서학개미 원픽’ 테슬라, 엇갈린 주가 전망",https://n.news.naver.com/mnews/article/009/000...,"[\n테슬라, 11거래일간 랠리 멈추고 급락주가 전망 엇갈려…‘로보택시’ 관건\n\...",2024-07-13 22:19:07
1,"""어라? 주가가 왜 이래""…'홈슬라' 기대하던 개미들 '멘붕'",https://n.news.naver.com/mnews/article/015/000...,"[\n""운임 급등에 '흠슬라' 재현되나 했더니""…목표가 하향 웬일? 하나증권 ""지속...",2024-07-13 21:31:09
2,하나만 알고 둘은 모른다더니…금투세의 비밀 [정경준의 주식어때],https://n.news.naver.com/mnews/article/215/000...,"[\n""내가 사면 떨어지고, 내가 팔면 오른다"". 당신만 모르는 주식투자의 불변의 ...",2024-07-13 21:01:00
3,"""전세계 돈 몰린다, 이 주식 담아라""…전문가들 '깜짝 전망' [신민경의 테마록]",https://n.news.naver.com/mnews/article/015/000...,"[\n증권가, 흔들리는 M7 ""여전히 담아라"" 외치는 이유 최근 매그니피센트 7 조...",2024-07-13 20:15:13
4,'무료 환전 시대' 환테크로 돈 벌어볼까…환테크 넘어 환투기 조짐까지,https://n.news.naver.com/mnews/article/057/000...,[\n\t\t\t【 앵커멘트 】 최근 엔화가 850원대까지 떨어지는 슈퍼 엔저 현...,2024-07-13 20:08:08
...,...,...,...,...
60,"""석유 싣던 탱크로리에 식용유 운송"" 중국서 경악한 사건, 주가도 폭삭",https://n.news.naver.com/mnews/article/008/000...,[\n[자오자오 차이나][편집자주] 중국은 가깝고도 먼 나라입니다. 서로를 의식하며...,2024-07-13 04:48:00
61,한달 남짓한 MSCI 지수 변경일…편입·편출설 나오는 종목은 어디,https://n.news.naver.com/mnews/article/008/000...,[\n\n\n\n\n8월 MSCI 한국지수 편출입 예상 종목/그래픽=이지혜 디자인 ...,2024-07-13 04:30:00
62,"수익은 기본, 따박따박 돈 쌓이는 재미도…좋은 배당주 4가지 기준 [부꾸미]",https://n.news.naver.com/mnews/article/008/000...,[\n김지영 베어링자산운용 펀드매니저정부의 밸류업 프로그램 발표 이후 배당주 투자에...,2024-07-13 03:30:00
63,"기술주 투매 뉴욕증시, 은행 실적 주목하며 반등 출발",https://n.news.naver.com/mnews/article/029/000...,[\n\t\t\t뉴욕 증시가 대형 기술주 투매 폭풍 후 주요 은행주 실적 발표에 주...,2024-07-13 00:43:06


In [6]:
def preprocess_text(text):
    # 불필요한 공백 및 개행 문자 제거
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    # 불필요한 특수 문자 제거
    text = re.sub(r'\[사진 출처.*?\]', '', text)
    text = re.sub(r'^\[|\]$', '', text)
    return text


In [8]:
# 'news_contents' 열 전처리
df['cleaned_news_contents'] = df['news_contents'].apply(preprocess_text)

print(df['cleaned_news_contents'][0])

 테슬라, 11거래일간 랠리 멈추고 급락주가 전망 엇갈려…‘로보택시’ 관건 국내 투자자들이 가장 많이 보유한 해외주식 테슬라 주가를 두고 엇갈린 전망이 나오고 있다. 입소문에 급등하는 ‘밈 주식’(유행성 주식)에 불과하다는 목소리가 있는 반면 테슬라의 상승 랠리가 또다시 이어질 것이란 분석도 나온다.13일 한국예탁결제원에 따르면 지난 10일 기준 해외 증시에 투자하는 서학개미들이 가장 많이 보유하고 있는 종목은 미국 전기차 대장주 테슬라로, 보관금액은 156억 2702만달러(21조5500억원) 규모다.테슬라 주가는 지난달 25일부터 11거래일 연속 이어온 상승 랠리를 끝으로 지난 11일(현지시간) 하락 전환했다. 다음달 8일로 예정됐던 자율주행차 ‘로보택시’(무인택시) 공개를 두 달 연기한다는 블룸버그의 보도가 악재로 작용한 것으로 풀이된다.최근 상승가도를 달린 테슬라 주가 강세의 배경으로는 월가 전문가 예상치를 웃도는 2분기 차량 인도(판매) 실적이 지목됐다.테슬라의 지난 2분기 인도량은 44만3956대로 전년동기대비 4.8% 감소했으나 시장 전망치를 소폭 상회했다. 시장의 우려보다 견조했던 전기차 판매량이 나오자 바닥을 확인했다는 인식이 퍼지며 매수세가 몰린 것으로 풀이된다.여기에 로보택시에 대한 기대감과 에너지저장장치(ESS) 사업을 영위하는 에너지 발전·저장 부문의 고성장도 주가 상승을 견인한 것으로 보인다.임은영 삼성증권 연구원은 “로보택시 기술 발표회를 앞두고 자율주행 기술에 대한 기대가 커졌다”며 “2분기를 포함해 하반기 실적도 부진할 것으로 예상돼 실적과 주가의 괴리가 커지는 구간”이라고 짚었다.테슬라는 올해 인공지능(AI) 랠리에서 소외된 데다 전기차 수요 둔화 우려와 경쟁 심화 등으로 연초 240달러선에서 지난 4월 140달러까지 무너졌다. 상승 랠리 전인 지난달 24일 기준으로 올 들어 26.50% 빠졌던 테슬라 주가는 연초 수준을 회복했다. 주가는 지난 11거래일 동안 40% 이상 뛰면서 엔비디아에 내줬던 해외주식 보관금액 1위 자리도 한

In [9]:
df['cleaned_news_contents']

0      테슬라, 11거래일간 랠리 멈추고 급락주가 전망 엇갈려…‘로보택시’ 관건 국내 투...
1      "운임 급등에 '흠슬라' 재현되나 했더니"…목표가 하향 웬일? 하나증권 "지속불가...
2      "내가 사면 떨어지고, 내가 팔면 오른다". 당신만 모르는 주식투자의 불변의 법칙...
3      증권가, 흔들리는 M7 "여전히 담아라" 외치는 이유 최근 매그니피센트 7 조정에...
4      【 앵커멘트 】 최근 엔화가 850원대까지 떨어지는 슈퍼 엔저 현상이 이어지며, ...
                            ...                        
60     [자오자오 차이나][편집자주] 중국은 가깝고도 먼 나라입니다. 서로를 의식하며 경...
61     8월 MSCI 한국지수 편출입 예상 종목/그래픽=이지혜 디자인 기자모간스탠리캐피털...
62     김지영 베어링자산운용 펀드매니저정부의 밸류업 프로그램 발표 이후 배당주 투자에 대...
63     뉴욕 증시가 대형 기술주 투매 폭풍 후 주요 은행주 실적 발표에 주목하며 반등세로...
64     효성화학 특수가스 사업부, 1.3조에 사모펀드 품으로VIG파트너스, 지분 매각 통...
Name: cleaned_news_contents, Length: 65, dtype: object